In [10]:
pip install Pillow torch torchvision torchaudio geopy requests SpeechRecognition pytesseract pandas numpy easyocr pyaudio pyttsx3 opencv-python pydub

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install easyocr torchvision torch requests pillow rouge

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import torch
import requests
import easyocr
import warnings
from PIL import Image
from datetime import datetime
from geopy.geocoders import Nominatim
from difflib import SequenceMatcher
import torchvision.transforms as transforms
import torchvision.models as models
import subprocess  # To open the file automatically

warnings.filterwarnings("ignore")

# ========== CONFIG ==========
API_KEY = "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"
MODEL_NAME = "google/gemma-3-27b-it:free"
API_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost",
    "X-Title": "EventReportGenerator"
}
# ============================

reader = easyocr.Reader(['en'], gpu=False)
geolocator = Nominatim(user_agent="event_report_generator")

# ========== INPUT ==========
def get_input_paths():
    data = {}
    data['event_name'] = input("Please enter the official name of the event: ").strip()
    data['college_name'] = input("Please enter the full name of the college/institution hosting the event: ").strip()

    images = []
    print("\nPlease provide paths to images related to the event (type 'done' when finished):")
    while True:
        path = input("Image path: ").strip()
        if path.lower() == "done":
            break
        if os.path.exists(path):
            images.append(path)
        else:
            print("❌ Invalid path.")
    if not images:
        print("⚠️ No valid images provided. The report will be generated without image context.")
    data['images'] = images

    if input("Would you like to attempt to recognize text in the images? (yes/no): ").lower().startswith('y'):
        data['recognize_text'] = True
    else:
        data['recognize_text'] = False

    if input("Is there any specific feedback or key information you'd like to ensure is mentioned in the report? (yes/no): ").lower().startswith('y'):
        data['feedback'] = input("Please provide the key feedback or information: ").strip()

    if input("Do you know the location of the event? (yes/no): ").lower().startswith('y'):
        data['location'] = input("Enter the location (city, state/region, country or latitude,longitude): ").strip()

    return data

# ========== OCR ==========
def process_image_text(path):
    try:
        results = reader.readtext(path)
        texts = [text for _, text, conf in results if conf > 0.6]
        return ", ".join(texts) if texts else None
    except Exception as e:
        print(f"⚠️ OCR processing error for {path}: {e}")
        return None

# ========== GEOLOCATION ==========
def get_college_from_location(loc):
    try:
        if "," in loc and all(part.strip().replace('.', '', 1).isdigit() for part in loc.split(',')):
            lat, lon = map(float, loc.split(','))
            place = geolocator.reverse((lat, lon), exactly_one=True, language='en')
        else:
            place = geolocator.geocode(loc, exactly_one=True, language='en')
        if place and 'address' in place.raw:
            addr = place.raw['address']
            potential_college = addr.get('university') or addr.get('college')
            if potential_college:
                return potential_college
            elif 'city' in addr and 'state' in addr:
                return f"{addr.get('city')}, {addr.get('state')}"
            else:
                return loc
        return loc
    except Exception as e:
        print(f"⚠️ Geolocation error for '{loc}': {e}")
        return loc

# ========== PROMPT ==========
def generate_prompt(data):
    event = data['event_name']
    college = data['college_name']
    location = get_college_from_location(data.get('location', '')) if 'location' in data else ''
    feedback = data.get('feedback', '')

    texts = []
    if data.get("recognize_text") and data.get('images'):
        for img in data['images']:
            ocr_text = process_image_text(img)
            if ocr_text:
                texts.append(f"Text found in images: '{ocr_text}'")

    prompt = (
        f"Generate a concise and factual report, presented in a single formal paragraph, that MUST include the following details:\n"
        f"- The official name of the event: {event}\n"
        f"- The full name of the host college/institution: {college}\n"
    )
    if 'location' in data and location:
        prompt += f"- The location of the event, specifying '{data['location']}' which is identified as being near '{location}'.\n"
    if texts:
        prompt += f"- Any relevant text observed in the images: {', '.join(texts)}.\n"
    if feedback:
        prompt += f"- The following key feedback or information: '{feedback}'.\n"
    prompt += "The report should be suitable for an institutional summary or a social media post. Ensure all provided details are accurately reflected in the report and avoid adding any external information or assumptions."

    return prompt

# ========== API CALL ==========
def send_prompt(prompt):
    try:
        res = requests.post(API_URL, headers=HEADERS, json={
            "model": MODEL_NAME,
            "messages": [{"role": "user", "content": prompt}]
        })
        if res.status_code == 200:
            out = res.json()
            return out['choices'][0]['message']['content'].strip()
        else:
            print(f"❌ API Error: {res.status_code} - {res.text}")
    except requests.exceptions.RequestException as e:
        print(f"❌ API Request Failed: {e}")
    except KeyError:
        print("❌ API Response Error: Could not extract report content.")
    return None

# ========== ACCURACY CHECK ==========
def check_accuracy(report, data):
    def normalize(text):
        return text.lower().strip()

    report_normalized = normalize(report)
    matched_keywords = 0
    total_relevant_inputs = 0

    if 'event_name' in data and data['event_name']:
        total_relevant_inputs += 1
        if normalize(data['event_name']) in report_normalized:
            matched_keywords += 1

    if 'college_name' in data and data['college_name']:
        total_relevant_inputs += 1
        if normalize(data['college_name']) in report_normalized:
            matched_keywords += 1

    if 'location' in data and data['location']:
        total_relevant_inputs += 1
        location_normalized_report = normalize(get_college_from_location(data['location'])) in report_normalized or normalize(data['location']) in report_normalized
        if location_normalized_report:
            matched_keywords += 1

    if 'feedback' in data and data['feedback']:
        total_relevant_inputs += 1
        feedback_keywords = normalize(data['feedback']).split()
        found_feedback_keywords = 0
        for keyword in feedback_keywords:
            if keyword in report_normalized:
                found_feedback_keywords += 1
        if feedback_keywords and (found_feedback_keywords / len(feedback_keywords)) > 0.4:  # Slightly more lenient for feedback
            matched_keywords += 1
        elif not feedback_keywords and normalize(data['feedback']) in report_normalized:
            matched_keywords += 1

    accuracy = round(matched_keywords / total_relevant_inputs, 2) if total_relevant_inputs > 0 else 1.0
    return accuracy

# ========== FILE HANDLING ==========
def save_and_open_report(report):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"report_{timestamp}.txt"
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"\n✅ Report saved to {filename}")
        # Automatically open the file based on the operating system
        if os.name == 'nt':  # Windows
            os.startfile(filename)
        elif os.name == 'posix':  # macOS and Linux
            subprocess.run(['open', filename])  # macOS
            # For Linux, you might need to try 'xdg-open' instead of 'open'
            # subprocess.run(['xdg-open', filename])
        else:
            print("⚠️ Could not automatically open the report. Please find it in the script's directory.")
    except Exception as e:
        print(f"❌ Error saving or opening the report: {e}")

# ========== MAIN ==========
def main():
    data = get_input_paths()
    if not data:
        return
    prompt = generate_prompt(data)
    print("\n--- Prompt Sent ---\n")
    report = send_prompt(prompt)
    if report:
        acc = check_accuracy(report, data)
        save_and_open_report(report)
        print(f"\n✅ Report Accuracy (based on keywords): {acc:.2f}/1.0")
    else:
        print("❌ Report generation failed.")

if __name__ == "__main__":
    main()

Using CPU. Note: This module is much faster with a GPU.



Please provide paths to images related to the event (type 'done' when finished):
⚠️ Geolocation error for 'Vignana Bharathi Institute of Technology, Office road, Ghatkesar mandal, Medchal–Malkajgiri, Telangana, 500088, India': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Vignana+Bharathi+Institute+of+Technology%2C+Office+road%2C+Ghatkesar+mandal%2C+Medchal%E2%80%93Malkajgiri%2C+Telangana%2C+500088%2C+India&format=json&limit=1&accept-language=en (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

--- Prompt Sent ---

❌ API Error: 401 - {"error":{"message":"No auth credentials found","code":401}}
❌ Report generation failed.
